In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim

### Run all models

In [2]:
sim.run(["build_networks"]) #initialize network accessibility engine
sim.run(["scheduled_development_events", "neighborhood_vars", #scheduled events and accessibility variables
         "rsh_simulate", "nrh_simulate", "nrh_simulate2",   #price models
         "jobs_transition", "elcm_simulate", "households_transition", "hlcm_luz_simulate", #demand/location models
         "price_vars", "feasibility", "residential_developer", "non_residential_developer", #supply/proforma models
         "model_integration_indicators" #indicators for pasef/pecas
         ], years=range(2013, 2041))

Running model 'build_networks'
Time to execute model 'build_networks': 13.76s
Total time to execute: 13.76s
Running year 2013
Running model 'scheduled_development_events'
Time to execute model 'scheduled_development_events': 0.87s
Running model 'neighborhood_vars'
Computing accessibility variables
Computing ave_parcel_size
Computing mean_nonres_rent_2000m
Computing jobs_1500m
Computing jobs_800m
Computing jobs_400m
Computing jobs1
Computing jobs2
Computing jobs3
Computing jobs4
Computing jobs5
Computing jobs6
Computing jobs7
Computing jobs8
Computing jobs9
Computing jobs10
Computing jobs11
Computing jobs12
Computing jobs13
Computing jobs14
Computing jobs15
Computing jobs16
Computing jobs17
Computing jobs18
Computing jobs19
Computing jobs20
Computing ave_income
Computing ave_age_of_head_1500m
Computing ave_children_1500m
Computing ave_year_built_1500m
Computing population_400m
Computing job_spaces_3000m
Computing jobs_3000m
Computing households_3000m
Computing residential_units_3000m
Co

c:\Anaconda\lib\site-packages\pandas\util\decorators.py:81: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  warnings.warn(msg, FutureWarning)
c:\Anaconda\lib\site-packages\pandas\util\decorators.py:13: FutureWarning: diff is deprecated. Use difference instead
  FutureWarning)



    and 0 overfull buildings
Time to execute model 'elcm_simulate': 34.53s
Running model 'households_transition'
Time to execute model 'households_transition': 17.13s
Running model 'hlcm_luz_simulate'
There are 100408 total movers for this LCM
HLCM for LUZ 1.0
There are 11314 total available units
    and 444 total choosers
    but there are 0 overfull buildings
    for a total of 4613 temporarily empty units
    in 3256 buildings total in the LUZ
Assigned 444 choosers to new units
Total currently unplaced: 99964
HLCM for LUZ 2.0
There are 12865 total available units
    and 509 total choosers
    but there are 0 overfull buildings
    for a total of 3654 temporarily empty units
    in 3239 buildings total in the LUZ
Assigned 509 choosers to new units
Total currently unplaced: 99455
HLCM for LUZ 3.0
There are 26708 total available units
    and 2114 total choosers
    but there are 0 overfull buildings
    for a total of 10030 temporarily empty units
    in 3416 buildings total in the

c:\Anaconda\lib\site-packages\pandas\core\index.py:670: FutureWarning: slice indexers when using iloc should be integers and not floating point
  "and not floating point",FutureWarning)
c:\Anaconda\lib\site-packages\pandas\computation\expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))



Unplaced jobs after: 915
Simulated DU: 46683.0
Target DU: 46006
Time to execute model 'residential_developer': 10.43s
Running model 'non_residential_developer'
768,166 feasible buildings before running developer
Job space target for heavy_industrial is -29036.5317706.
Job space target for light_industrial is -156053.17815.
Job space target for retail is 100342.86.
Sum of net units that are profitable: 1,265,786
Constructed 1118 retail buildings, totaling 102910.848546 new job spaces.
Unplaced households before: 4510
Unplaced households after: 8192
Unplaced jobs before: 915
Unplaced jobs after: 1638
Job space target for office is 5306.71581.
Sum of net units that are profitable: 810,556
Constructed 34 office buildings, totaling 6253.17247092 new job spaces.
Unplaced households before: 8192
Unplaced households after: 8408
Unplaced jobs before: 1638
Unplaced jobs after: 2236
Time to execute model 'non_residential_developer': 7.39s
Running model 'model_integration_indicators'
Exporting in

In [ ]:
# LUZ prices
bsim = sim.get_table('buildings').to_frame(columns = ['luz_id', 'note', 'res_price_per_sqft', 'nonres_rent_per_sqft', 'residential_units', 'non_residential_sqft'])
luz_res_price = bsim[bsim.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = bsim[bsim.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
bsim = bsim[bsim.note == 'simulated']
luz_simdu = bsim.groupby('luz_id').residential_units.sum()
luz_simnr = bsim.groupby('luz_id').non_residential_sqft.sum()
luz_df = pd.DataFrame({'du':luz_simdu, 'nrsf':luz_simnr, 'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
# luz_df.to_csv(loader.get_path('out/luz_sim_indicators.csv'))

In [5]:
# Summarize results at MSA level
b = sim.get_table('buildings').to_frame(columns = ['msa_id', 'mgra_id', 'residential_units', 'non_residential_sqft', 'note'])
new_du_by_msa = b[b.note  == 'simulated'].groupby('msa_id').residential_units.sum()
new_nrsf_by_msa = b[b.note  == 'simulated'].groupby('msa_id').non_residential_sqft.sum()
proportion_du_by_msa = new_du_by_msa / new_du_by_msa.sum()
proportion_nrsf_by_msa = new_nrsf_by_msa / new_nrsf_by_msa.sum()

In [6]:
print proportion_du_by_msa
print proportion_nrsf_by_msa 

msa_id
0         0.092584
1         0.385550
2         0.348743
3         0.013451
4         0.107693
5         0.051979
Name: residential_units, dtype: float64
msa_id
0         0.122078
1         0.588730
2         0.121997
3         0.001476
4         0.129701
5         0.036019
Name: non_residential_sqft, dtype: float64


### Predict prices and summarize/export at the LUZ level

In [ ]:
%%capture
## Accessibility engine
sim.run(["build_networks", "neighborhood_vars"])

# Predict prices
sim.run(["rsh_simulate", "nrh_simulate", "nrh_simulate2"])

##Export LUZ mean prices to csv for visual checking
buildings = sim.get_table('buildings').to_frame()
luz_res_price = buildings[buildings.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = buildings[buildings.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
df = pd.DataFrame({'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
# df.fillna(0).to_csv(loader.get_path('out/sim/sd_prices.csv'))

# Summarize prices using network queries
sim.run(['price_vars'])

### Proforma

In [ ]:
# from urbansim.developer import sqftproforma
# # Step 1:  feasibility
sim.run(['feasibility'])
sim.run(["residential_developer"])
sim.run(["non_residential_developer"])